<a href="https://colab.research.google.com/github/sunflowcoco/SS_Classification/blob/master/CNN_SS_320.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
SS 320

自己的方法实现



'''

from __future__ import print_function
import keras
import sys
from keras.datasets import mnist

from keras.models import Sequential

from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop

from keras import backend as K

import numpy as np  
import os
import time

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score,accuracy_score

np.random.seed(1337)
from keras.utils import np_utils  
from keras.models import Model

#DNN setting
batch_size = 256
num_classes = 6
epochs = 100
#input image dimensions
# img_rows, img_cols = 20, 16
img_rows, img_cols = 8, 40



###############################################################################
#If equall to 'yes', then load from Bin file
FAST_LOAD = 'yes'
#postfix
#FILE_TYPE = '21_40d_62851r_122010'#'756d_less'#'1134d'
FILE_TYPE='21_40d_104685r_12316_VPN'
#Feature column
COLUMN = 320
#K-Fold
K = 3
k = 1
# if len(sys.argv)>=2:
# 	k = int(sys.argv[1])
# if len(sys.argv)>=3:
# 	K = int(sys.argv[2])	


path = '/content/drive/SS/'

#Log setting
LOG_FLAG = 1
log_str = ''
# logfilename = '../log/'+os.path.basename(sys.argv[0])+'_'+str(K)+'_'+str(k)+'.txt'
logfilename = path+ '/log/'+os.path.basename(sys.argv[0])+'_'+str(K)+'_'+str(k)+'.txt'

log_str = log_str + 'Start: '+time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+'\r\n'
	
# src_file = '../data/sample_combinnativeflat21_40d_134685r_12522.txt'
# bin_file = '../data/sample_combinnativeflat21_40d_134685r_320.bin'

# src_file =path + 'data/sample_combinnativeflat21_40d_134685r_12522.txt'
# bin_file=path + 'data/sample_combinnativeflat21_40d_104685r_800.bin'

src_file =path + '/data/sample_combinnativeflat21_40d_104685r_12711.txt'
bin_file=path + '/data/sample_combinnativeflat21_40d_104685r_320.bin'


sample = []
if FAST_LOAD=='yes':
	sample = np.fromfile(bin_file, dtype=np.double)  
	sample = sample.reshape(-1, COLUMN+1)
else:
	orig = (np.loadtxt(src_file, dtype=np.str, delimiter=","))[:,40:360].astype(np.double).reshape(-1, 320)
	label = (np.loadtxt(src_file, dtype=np.str, delimiter=","))[:,-1].astype(np.double).reshape(-1, 1)-2
	
	print(orig.shape)
	print(label.shape)
	
	sample = np.hstack((orig, label))
	np.random.shuffle(sample)
	sample.tofile(bin_file)



gridLen = int(len(sample)*1.0/K)
s_index = (k-1)*gridLen
e_index = k*gridLen

testX = sample[s_index:e_index, :-1]*255
testY = sample[s_index:e_index, -1]


if k==1:
	trainX = sample[e_index:, :-1]*255
	trainY = sample[e_index:, -1]
elif k==K:
	trainX = sample[0:s_index, :-1]*255
	trainY = sample[0:s_index, -1]
else:	
	trainX = np.vstack((sample[0:s_index, :-1], sample[e_index:, :-1]))*255
	trainY = np.hstack((sample[0:s_index, -1], sample[e_index:, -1]))

trainYBackup = trainY
testYBackup = testY

print(trainX.max())
print(trainX.min())

del sample

print ('Train Size:'+str(len(trainY)))
print ('Test Size:'+str(len(testY)))

255.0
0.0
Train Size:69790
Test Size:34895


In [0]:

###############################################################################

#save loss acc
class LossHistory(keras.callbacks.Callback):
	def on_train_begin(self, logs={}):
		self.losses = {'batch':[], 'epoch':[]}
		self.accuracy = {'batch':[], 'epoch':[]}
		self.val_loss = {'batch':[], 'epoch':[]}
		self.val_acc = {'batch':[], 'epoch':[]}

	def on_batch_end(self, batch, logs={}):
		self.losses['batch'].append(logs.get('loss'))
		self.accuracy['batch'].append(logs.get('acc'))
		self.val_loss['batch'].append(logs.get('val_loss'))
		self.val_acc['batch'].append(logs.get('val_acc'))

	def on_epoch_end(self, batch, logs={}):
		self.losses['epoch'].append(logs.get('loss'))
		self.accuracy['epoch'].append(logs.get('acc'))
		self.val_loss['epoch'].append(logs.get('val_loss'))
		self.val_acc['epoch'].append(logs.get('val_acc'))

	def getHistory(self):
		epoch_losses_str = ','.join(str(i) for i in self.losses['epoch'])
		epoch_accuracy_str = ','.join(str(i) for i in self.accuracy['epoch'])
		epoch_val_loss_str = ','.join(str(i) for i in self.val_loss['epoch'])
		epoch_val_acc_str = ','.join(str(i) for i in self.val_acc['epoch'])
		
		batch_losses_str = ','.join(str(i) for i in self.losses['batch'])
		batch_accuracy_str = ','.join(str(i) for i in self.accuracy['batch'])
		#batch_val_loss_str = ','.join(str(i) for i in self.val_loss['batch'])
		#batch_val_acc_str = ','.join(str(i) for i in self.val_acc['batch'])
		
		rt_str = ''
		rt_str = rt_str + 'epoch_losses:' + epoch_losses_str + '\r\n'
		rt_str = rt_str + 'epoch_accuracy:' + epoch_accuracy_str + '\r\n'
		rt_str = rt_str + 'epoch_val_loss:' + epoch_val_loss_str + '\r\n'
		rt_str = rt_str + 'epoch_val_acc:' + epoch_val_acc_str + '\r\n'
		
		rt_str = rt_str + 'batch_losses:' + batch_losses_str + '\r\n'
		rt_str = rt_str + 'batch_accuracy:' + batch_accuracy_str + '\r\n'
		#rt_str = rt_str + 'batch_val_loss:' + batch_val_loss_str + '\r\n'
		#rt_str = rt_str + 'batch_val_acc:' + batch_val_acc_str + '\r\n'
		
		return rt_str
		
###############################################################################
#history = LossHistory()
	
	
trainX = trainX.reshape(-1, img_rows, img_cols, 1)
testX = testX.reshape(-1, img_rows, img_cols, 1)
print('trainX.shape: ',trainX.shape)

trainY=np_utils.to_categorical(trainY, num_classes)
testY=np_utils.to_categorical(testY, num_classes)

input_shape = (img_rows, img_cols, 1)


from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='bestmodel_yyw_320.h5',monitor='val_loss',verbose=1,save_best_only=True)


input_data = Input(shape=input_shape)
conv = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_data)
# conv = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Conv2D(64, (5, 5), activation='relu')(conv)
conv = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Flatten()(conv)
conv_output = Dense(128, activation='relu')(conv)
full = Dense(64, activation='relu')(conv_output)
full = Dropout(0.4)(full)
full_output = Dense(num_classes, activation='softmax')(full)

model = Model(inputs=input_data, outputs=full_output)


model.summary()

# model.compile(loss=keras.losses.categorical_crossentropy,
# 			  optimizer=keras.optimizers.Adadelta(),
# 			  metrics=['accuracy'])
rmsprop=RMSprop(lr=0.001,rho=0.9,epsilon=1e-08,decay=0.00) 
model.compile(loss=keras.losses.categorical_crossentropy,
			  optimizer=rmsprop,
			  metrics=['accuracy'])

history=model.fit(trainX, trainY,
		  batch_size=batch_size,
		  epochs=epochs,
		  verbose=1,
		  validation_data=(testX, testY),
		  callbacks=[checkpoint])


#model.save(os.path.basename(sys.argv[0])+'_model'+'_'+FILE_TYPE+'.h5')
test_rst = model.predict(testX)

test_label = np.argmax(test_rst, axis=1).astype(np.int32)
#confusion_matrix = np.zeros((num_classes,num_classes))
testY = testYBackup.astype(np.int32)

print(history.history,file=open('history_320','w'))
cf=confusion_matrix(testY,test_label)


pre=precision_score(testY,test_label,average=None)
rec=recall_score(testY,test_label,average=None)
f1_s=f1_score(testY,test_label,average=None)

# print(cf,file=open('confusion_matrix_320','w'))
# print(pre,file=open('precision_320','w'))
# print(rec,file=open('recall_320','w'))
# print(f1_s,file=open('f1_score_320','w'))

print(cf)
print(pre)
print(rec)
print(f1_s)

acc=accuracy_score(testY,test_label)
print(acc)

trainX.shape:  (69790, 8, 40, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 8, 40, 1)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 38, 32)         320       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 2, 34, 64)         51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 1, 17, 64)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1088)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               139392    
_________________________________________________________________
dense_8 (Dense)              (None, 64)    